# Image Processing Module

## Errors

In [1]:
'''Custom defined errors for this package'''
class ArgumentValidationError(ValueError):
    '''
    Raised when the type of an argument to a function is not what it should be.
    '''
    def __init__(self, arg_num, func_name, accepted_arg_type):
        self.error = 'The {0} argument of {1}() is not a {2}'.format(arg_num,
                                                                     func_name,
                                                                     accepted_arg_type)
        super(ArgumentValidationError, self).__init__(arg_num, func_name, accepted_arg_type)

    def __str__(self):
        return self.error

class InvalidArgumentNumberError(ValueError):
    '''
    Raised when the number of arguments supplied to a function is incorrect.
    Note that this check is only performed from the number of arguments
    specified in the validate_accept() decorator. If the validate_accept()
    call is incorrect, it is possible to have a valid function where this
    will report a false validation.
    '''
    def __init__(self, func_name):
        self.error = 'Invalid number of arguments for {0}()'.format(func_name)
        super(InvalidArgumentNumberError, self).__init__(func_name)

    def __str__(self):
        return self.error

class InvalidReturnType(ValueError):
    '''
    As the name implies, the return value is the wrong type.
    '''
    def __init__(self, return_type, func_name):
        self.error = 'Invalid return type {0} for {1}()'.format(return_type,
                                                                func_name)
        super(InvalidReturnType, self).__init__(return_type, func_name)

    def __str__(self):
        return self.error


## Helpers

In [2]:
''' General support functions that are helpful in managing logic
across applications.
'''
#pylint: disable=W0613, C0301
import functools
import re
from imageprocessing.errors import InvalidArgumentNumberError
from imageprocessing.errors import ArgumentValidationError
from imageprocessing.errors import InvalidReturnType

REG = re.compile(r'(?!^)(?<!_)([A-Z])')

def camel_to_snake(s):
    '''
    Converts a string from CamelCase to snake_case.
    '''
    return REG.sub(r'_\1', s).lower()

def ordinal(num):
    '''
    Returns the ordinal number of a given integer, as a string.
    eg. 1 -> 1st, 2 -> 2nd, 3 -> 3rd, etc.
    '''
    if 10 <= num % 100 < 20:
        return '{0}th'.format(num)

    ordn = {1 : 'st', 2 : 'nd', 3 : 'rd'}.get(num % 10, 'th')
    return '{0}{1}'.format(num, ordn)

def accepts(*accepted_arg_types):
    '''
    A decorator to validate the parameter types of a given function.
    It is passed a tuple of types. eg. (<type 'tuple'>, <type 'int'>)

    Note: It doesn't do a deep check, for example checking through a
          tuple of types. The argument passed must only be types.
    '''

    def accept_decorator(validate_function):
        '''Check if the number of arguments to the validator
        function is the same as the arguments provided
        to the actual function to validate. We don't need
        to check if the function to validate has the right
        amount of arguments, as Python will do this
        automatically (also with a TypeError).
        '''
        functools.wraps(validate_function)
        def decorator_wrapper(*function_args, **function_args_dict):
            '''Enumerate over arguments to check for types.
            We're using enumerate to get the index, so we can pass the
            argument number with the incorrect type to ArgumentValidationError.
            '''
            if len(accepted_arg_types) is not len(accepted_arg_types):
                raise InvalidArgumentNumberError(validate_function.__name__)

            for arg_num, (actual_arg, accepted_arg_type) in enumerate(zip(function_args, accepted_arg_types)):
                if not isinstance(actual_arg) is accepted_arg_type:
                    print(actual_arg, accepted_arg_types)
                    ord_num = ordinal(arg_num + 1)
                    raise ArgumentValidationError(ord_num,
                                                  validate_function.__name__,
                                                  accepted_arg_type)

            return validate_function(*function_args)
        return decorator_wrapper
    return accept_decorator

def returns(*accepted_return_type_tuple):
    '''
    Validates the return type. Since there's only ever one
    return type, this makes life simpler. Along with the
    accepts() decorator, this also only does a check for
    the top argument. For example you couldn't check
    (<type 'tuple'>, <type 'int'>, <type 'str'>).
    In that case you could only check if it was a tuple.
    '''
    def return_decorator(validate_function):
        '''Process the decorator'''
        # No return type has been specified.
        #pylint: disable=C1801
        if len(accepted_return_type_tuple) == 0:
            raise TypeError('You must specify a return type.')

        functools.wraps(validate_function)
        def decorator_wrapper(*function_args):
            '''Process arguments to determine if the are the
            correct return type'''
            # More than one return type has been specified.
            if len(accepted_return_type_tuple) > 1:
                raise TypeError('You must specify one return type.')

            # Since the decorator receives a tuple of arguments
            # and the is only ever one object returned, we'll just
            # grab the first parameter.
            accepted_return_type = accepted_return_type_tuple[0]

            # We'll execute the function, and
            # take a look at the return type.
            return_value = validate_function(*function_args)
            return_value_type = type(return_value)

            if return_value_type is not accepted_return_type:
                raise InvalidReturnType(return_value_type,
                                        validate_function.__name__)

            return return_value

        return decorator_wrapper
    return return_decorator


## Name Generator

In [3]:
'''Generate a file name from Image props.  Mostly used to allow for 
more complex descriptive naming schemes such as 1-1-rotate_90-contrast_0.2.png
'''
import sys
from imageprocessing.image_props import ImageProps
from imageprocessing.font_props import FontProps

class NameGenerator(object):
    '''Generate a file name from Image props.  Mostly used to
    allow for more complex descriptive naming schemes such as
    1-1-rotate_90-contrast_0.2.png
    '''
    # pylint: disable=R0913
    # pylint: disable=R0902

    def __init__(self, klass=0, image_props=ImageProps(), font_props=FontProps(), name='UNAMED.png'):
        self._klass= klass
        self._image_props = image_props
        self._font_props = font_props
        self._name = name

    def __eq__(self, other):
        '''Overrides the default implementation'''
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        '''Overrides the default implementation (unnecessary in Python 3)'''
        return not self.__eq__(other)

    @property
    def font_props(self):
        '''The font propeties.  If none included, will
        inialize a default FontProps object
        '''
        if not self._font_props:
           self._font_props = FontProps()

        return self._font_props

    @property
    def image_props(self):
        '''The image properties used to generate the name. If none
        this will initialize a new ImageProps object to use
        its default properties,  and provide a default file name
        `original` suffix.
        '''
        if not self._image_props:
            self._image_props = ImageProps()
        return self._image_props

    @property
    def name(self):
        '''The generated name from based on the font and image properties.
       '''
        self._name = "%s-%s-%s" % (self.image_props.klass,
                                       self._klass, self._image_props.name)
        return self._name

## Font Property Model

In [4]:
'''Convenience class to store properties about fonts that will be used
to generate image data sets'''
import os
from imageprocessing.helpers import camel_to_snake
class FontProps(object):
    '''A convenience class that provides properties to use modifying fonts for images objects'''
    def __init__(self,
                 color=(0, 0, 0, 255),
                 size=27,
                 font_path=unicode("/System/Library/Fonts/HelveticaNeue.ttc"),
                ):
        self._color = color
        self._size = size
        self._font_path = font_path 

    def __eq__(self, other):
        '''Overrides the default implementation'''
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        '''Overrides the default implementation (unnecessary in Python 3)'''
        return not self.__eq__(other)

    @property
    def color(self):
        '''Font color'''
        return self._color

    @property
    def size(self):
        '''Size of the font'''
        return self._size

    @property
    def font_path(self):
        '''Unicode file path location'''
        return self._font_path

    def filename(self):
        '''Unicode file path location'''
        return os.path.basename(self._font_path)

    def name (self):
        '''Returns the basename of the font with the extension'''
        (_name, _ext) = os.path.splitext(self.filename())
        return _name

    def foldername (self):
        '''Returns the name of the font.  It lowercases the string,
        strips out any spaces and replaces with "-" '''
        (_name, _ext) = os.path.splitext(self.filename())
        return _name.replace(" ", "-").lower()

    def __str__(self):
        attrs = vars(self)
        return '\n'.join("%s: %s" % item for item in attrs.items())
    

## Image Props

In [5]:
'''A convenience class that provides properties to use in transforming images objects'''
class ImageProps(object):
    '''A convenience class that provides properties to use in transforming images objects'''

    # pylint: disable=R0913
    # pylint: disable=R0902
    def __init__(self,
                 name='original',
                 rotation=0,
                 background_color=(255, 255, 255, 255),
                 text_color=(0, 0, 0, 255),
                 contrast=1.0,
                 brightness=1.0,
                 color_adjust=1.0,
                 sharpness=1.0,
                 size=(28, 28),
                ):
        self._name = name
        self._rotation = rotation
        self._background_color = background_color
        self._text_color = text_color
        self._contrast = contrast
        self._color_adjust = color_adjust
        self._sharpness = sharpness
        self._brightness = brightness
        self._size = size

    def __eq__(self, other):
        '''Overrides the default implementation'''
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        '''Overrides the default implementation (unnecessary in Python 3)'''
        return not self.__eq__(other)

    @property
    def name(self):
        '''Name of the image property state'''
        return self._name

    @property
    def rotation(self):
        '''degrees of rotation, counter clockwise'''
        return self._rotation

#     @rotation.setter
#     def rotation(self, value):
#         self._rotation = value

    @property
    def text_color(self):
        '''Color the text will be rendered on the image'''
        return self._text_color

    @text_color.setter
    def text_color(self, value):
        self._text_color = value

    @property
    def background_color(self):
        '''Background color on the image'''
        return self._background_color

#     @background_color.setter
#     def background_color(self, value):
#         self._background_color = value

    @property
    def contrast(self):
        '''Change contrast of image from 0.0...1.0'''
        return self._contrast

#     @contrast.setter
#     def contrast(self, value):
#         self._contrast = value

    @property
    def sharpness(self):
        '''Change sharepness of image from 0.0...1.0....2.0'''
        return self._sharpness

#     @sharpness.setter
#     def sharpness(self, value):
#         self._sharpness = value

    @property
    def brightness(self):
        '''Change brigthness of image from 0.0...1.0'''
        return self._brightness

#     @brightness.setter
#     def brightness(self, value):
#         self._brightness = value

    @property
    def color_adjust(self):
        '''Change brigthness of image from 0.0...1.0'''
        return self._color_adjust

#     @color.setter
#     def color(self, value):
#         return self._color

    @property
    def size(self):
        '''Tuple (width, height) Size of image '''
        return self._size

#     @size.setter
#     def size(self, value):
#         return self._size

    def __str__(self):
        attrs = vars(self)
        return '\n'.join("%s: %s" % item for item in attrs.items())
    

## Image Generator

In [6]:
'''A custom convenience class to support generating image datasets for use with Pillow'''
import os
from fontconfig import FcFont
from imageprocessing.helpers import accepts
from imageprocessing.image_props import ImageProps
from imageprocessing.font_props import FontProps
from PIL import Image


class ImageGenerator(object):
    '''A custom convenience class to support generating image datasets for use with Pillow'''

    # pylint: disable=R0913
    # pylint: disable=R0902

    def __init__(self,
                 text='A',
                 klass=0,
                 image_id=0,
                 image_props=None,
                 image=None,
                 output_dir='data',
                 filename='UNAMED.png'):
        self._text = text
        self._klass = klass
        self._fontname = klass
        self._image_id = image_id
        self._image = image
        self._image_props = image_props
        self._output_dir = output_dir
        self._filename = filename
        self._saved = False

    def __eq__(self, other):
        '''Overrides the default implementation'''
        if isinstance(self, other.__class__):
            return self.__dict__ == other.__dict__
        return False

    def __ne__(self, other):
        '''Overrides the default implementation (unnecessary in Python 3)'''
        return not self.__eq__(other)

    @property
    def text(self):
        '''The text to be rendered on the image. Probably should be a letter'''
        return self._text

    @property
    def filename(self):
        '''The name of the file. This will initialize
        a new ImageProps object to use its default properties,
        and provide a default file name `original` suffix.
        '''
        if not self._image_props:
            self._image_props = ImageProps()

        print self.image_props
        self._filename = "%s-%s-%s" % (self._klass,
                                       self._image_id, self._image_props.name)
        if self._filename == "0-0-original":
            print "This is the first image in the series! [0-0-original]"
        return self._filename

    @property
    def output_dir(self):
        '''The output directory where the generated images
        will be stored. Used in the saving and naming of the file.
        '''
        return self._output_dir

    @property
    def klass(self):
        '''Class identifier used to map to a specific character.
        'A' should always be 0. Used in the naming of the file.'''
        return self._klass

    @property
    def image_props(self):
        '''Image id - should be incremented for each image generated. Used
        in the naming of the file.'''
        return self._image_props

    @image_props.setter
    def image_props(self, value):
        '''ImageProp object.  This object must be of type ImageProps'''
        if not isinstance(value, list):
            raise TypeError("image_props must be a list of type ImageProps")

        if value == self._image_props:
            raise ValueError('''Incoming image_props is identical to
                             current! Are you sure you want that to happen?''')

        if self._saved:
            self._saved = False

        self._image_props = value

    @property
    def image_id(self):
        '''Image id - should be incremented for each image generated. Used
        in the naming of the file.'''
        return self._image_id

    @property
    def image(self):
        '''Returns the test object image.  `image` is
        of type Pil.Image'''

        return self._image



    @image.setter
    def image(self, value):
        '''Sets the image to be used.  This should normally be set
        during initialization, but this is here for convenience. The
        value must be an instance of PIL.Image.'''
        if not isinstance(value, Image):
            raise TypeError("Image must be of type PIL.Image")
        self._image = value

    @property
    def saved(self):
        '''Convenience value to prevent images being saved twice.'''
        return self._saved

    
    def save(self):
        '''Saves the test data object.  This will call the filename property
        in order to grab the class, image id, and image_propss object.  Since
        this calls `filename`, an instance of the default image_propss object
        is created if there isn't one'''
        if self._image and not self._saved:
            self._image.save(unicode(self._filename + ".png"))
            self._saved = True
            print "SAVED IMAGE"
        elif self._saved:
            print "Already saved this image - overwriting not implemented"
        else:
            raise IOError(
                "Unable to save image file - it doesn't exist!  Add a PIL Image object")

    @staticmethod
    def generate_images_by_font(
            font=None,
            characters,
            image_props=None):
        '''Statically called method that takes a list of characters, generates the data set
        This is equivalent to a main processing function and is the entry point for outside
        integration'''
        if not font and not isinstance(font, fontconfig):
            raise TypeError("font must be of type FontProps")

        if not image_props and not isinstance(image_props, ImageProps):
            raise TypeError("image_props must be of type ImageProps")

        image_index = 0

        print "\nGenerating images for these characters: \n\n{0}\n\n".format(characters)
        # print "Using the following fonts: \n"
        # print '\n'.join([str(item) for item in fonts])
        klass = 0
        # print ','.join([str(item) for item in characters])

        ## START PROCESS
        for character in characters:
            if not isinstance(character, str) and not isinstance(character, int):
                raise TypeError('Must be of type str or integer ')
            print "{0}-{1}-{2}-{3}.png".format(klass, character, image_index, image_props.name)
            ## cycle through fonts and create font objects
            # for image_prop in image_props:
            #     generator = ImageGenerator(
            #         text=str(character),
            #         klass=klass,
            #         image_id=image_index,
            #         image_props=image_props,
            #     )
            #     print generator.filename
            # print "\nCreated the following generator: \n{0}\n".format(generator)
            image_index += 1
            # end - image should be saved and we incremented the image_index id
            # end character loop - we should have images that
            # are of different fonts with different transformations
            # of a specific character
            klass += 1

    @accepts(str, FcFont, ImageProps)
    @staticmethod
    def generate_images_by_character(
            character,
            font=None,
            image_props=None):
        '''Statically called method that takes a list of characters, generates the data set
        This is equivalent to a main processing function and is the entry point for outside
        integration'''
        if not font and not isinstance(font, FcFont):
            raise TypeError("font must be of type FontProps")

        if not image_props and not isinstance(image_props, ImageProps):
            raise TypeError("image_props must be of type ImageProps")

        image_index = 0

        print "\nGenerating images for these characters: \n\n{0}\n\n".format(characters)
        # print "Using the following fonts: \n"
        # print '\n'.join([str(item) for item in fonts])
        klass = 0
        # print ','.join([str(item) for item in characters])

        ## START PROCESS
        for character in characters:
            if not isinstance(character, str) and not isinstance(character, int):
                raise TypeError('Must be of type str or integer ')
            print "{0}-{1}-{2}-{3}.png".format(klass, character, image_index, image_props.name)
            ## cycle through fonts and create font objects
            # for image_prop in image_props:
            #     generator = ImageGenerator(
            #         text=str(character),
            #         klass=klass,
            #         image_id=image_index,
            #         image_props=image_props,
            #     )
            #     print generator.filename
            # print "\nCreated the following generator: \n{0}\n".format(generator)
            image_index += 1
            # end - image should be saved and we incremented the image_index id
            # end character loop - we should have images that
            # are of different fonts with different transformations
            # of a specific character
            klass += 1

    def __str__(self):
        attrs = vars(self)
        desc = '\n'.join("%s: %s" % item for item in attrs.items())
        return desc


SyntaxError: non-default argument follows default argument (<ipython-input-6-706954f8bef8>, line 146)

### Test

In [ ]:
# -*- encoding: utf-8 -*-
'''Main Processing Logic'''
# from imageprocessing.helpers import accepts, returns
import sys
import fontconfig
from imageprocessing import FontProps, ImageProps, ImageGenerator
# Generate our fonts and image sets
IMAGE_ROTATE_45     = ImageProps(name='rotate_45', rotation=45)
IMAGE_ROTATE_90     = ImageProps(name='rotate_90', rotation=90)
IMAGE_ROTATE_135    = ImageProps(name='rotate_135', rotation=135)
IMAGE_ROTATE_180    = ImageProps(name='rotate_180', rotation=180)
IMAGE_ROTATE_225    = ImageProps(name='rotate_225', rotation=225)
IMAGE_ROTATE_270    = ImageProps(name='rotate_270', rotation=270)
IMAGE_ROTATE_315    = ImageProps(name='rotate_315', rotation=315)

IMAGE_CONTRAST_015  = ImageProps(name='contrast_0_15', contrast=0.15)
IMAGE_CONTRAST_025  = ImageProps(name='contrast_0_25', contrast=0.25)
IMAGE_CONTRAST_035  = ImageProps(name='contrast_0_35', contrast=0.35)
IMAGE_CONTRAST_045  = ImageProps(name='contrast_0_45', contrast=0.45)
IMAGE_CONTRAST_055  = ImageProps(name='contrast_0_55', contrast=0.55)
IMAGE_CONTRAST_065  = ImageProps(name='contrast_0_65', contrast=0.65)

IMAGE_COLOR_080     = ImageProps(name='color_80', background_color=(80, 80, 80, 155))
IMAGE_COLOR_096      = ImageProps(name='color_96', background_color=(96, 96, 96, 155))
IMAGE_COLOR_128     = ImageProps(name='color_120', background_color=(120, 120, 120, 255))
IMAGE_COLOR_128     = ImageProps(name='color_128', background_color=(128, 128, 128, 255))
IMAGE_COLOR_128     = ImageProps(name='color_128', background_color=(0, 204, 128, 68))

IMAGE_ROTATED = [
    IMAGE_ROTATE_45,
    IMAGE_ROTATE_90,
    IMAGE_ROTATE_135,
    IMAGE_ROTATE_180,
    IMAGE_ROTATE_225,
    IMAGE_ROTATE_270,
    IMAGE_ROTATE_315,

]
IMAGE_CONTRAST = [
    IMAGE_CONTRAST_015,
    IMAGE_CONTRAST_025,
    IMAGE_CONTRAST_035,
    IMAGE_CONTRAST_045,
    IMAGE_CONTRAST_055,
    IMAGE_CONTRAST_065,
]
IMAGE_COLOR = [
    IMAGE_COLOR_080,
    IMAGE_COLOR_096,
    IMAGE_COLOR_128,
    IMAGE_COLOR_128,
    IMAGE_COLOR_128,
]

IMAGE_PROPS_LIST = IMAGE_ROTATED + IMAGE_CONTRAST + IMAGE_COLOR
# pylint: disable=E1101
ALL_FONTS = fontconfig.query(lang='en')

def clean_fonts(fonts):
    '''Clean up bad fonts that have non ascii characters - no need to use them'''
    try:
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W0.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ明朝 ProN.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ明朝 ProN.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ丸ゴ ProN W4.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W1.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W2.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W5.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W6.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W6.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W7.ttc") 
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W8.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W8.ttc")
        fonts.remove(u"/System/Library/Fonts/ヒラギノ角ゴシック W9.ttc")
    except ValueError:
        pass # or scream: thing not in some_list!
    except AttributeError:
        print "Unexpected error:", sys.exc_info()[0]
        pass

clean_fonts(ALL_FONTS)

print "\n{0} fonts to be processed.\n".format(len(ALL_FONTS))

FONT_PROPS_LIST = list()
for font in ALL_FONTS:
    FONT_PROPS_LIST.append(FontProps(font_path=font))

# Character set that will be readable
# pylint: disable=C0301
UPPERCASE = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
LOWERCASE = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
NUMBERS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
CHARACTERS = ["'", ","]

READABLE_CHARACTERS = UPPERCASE + LOWERCASE + NUMBERS + CHARACTERS

TEST_CHARACTERS = ['A']
ImageGenerator.process_characters(
    READABLE_CHARACTERS,
    fonts=FONT_PROPS_LIST,
    image_props=IMAGE_PROPS_LIST
)
# ImageGenerator.process_characters(TEST_CHARACTERS, ALL_FONTS, IMAGEPROP1)
print 'DONE'


### Supported Characters

In [ ]:
uppercase=[ 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
lowercase=[ 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z' ]
numbers=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
characters=[ "'", "," ]
test_characters = uppercase + lowercase + numbers + characters

In [ ]:
ImageGenerator.process_characters(['A'], [PROPS], [])